In [ ]:
import numpy as np
import pandas as pd
from mioni_helper import load_mioni_data
from random_helper_functions import get_bitstring

In [ ]:
import sys

import matplotlib.pyplot as plt

sys.path.append('../..')
from plotting.matplotlib_setup import configure_latex, savefig, set_size_decorator, savefig, thiner_border

tex_dir, images_dir = 'porocilo/main.tex', 'porocilo/images'

configure_latex(style=['science', 'notebook'], global_save_path=images_dir)

%config InlineBackend.figure_format = 'pdf'

# Meritve

In [ ]:
dfs = load_mioni_data()

In [ ]:
for df in dfs:
    ind = df[np.abs(df['N']) < 1].index
    df.drop(ind, inplace=True)
    
    ind = df[df['us'] < 0].index
    df.drop(ind, inplace=True)

In [ ]:
plt.yscale('log')

for df in dfs:
    plt.plot(df['us'], np.abs(df['N']))

In [ ]:
dfs = [dfs[0], dfs[2]]

In [ ]:
dfs[0] = dfs[0][(len(dfs[0]) - len(dfs[1])):]

In [ ]:
fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)

ax.set_yscale('log')

for df in dfs:
    ax.plot(df['us'], np.abs(df['N']), lw=1)

ax.set_xlabel(r'$t$ [$\mu$s]')
ax.set_ylabel('$N$')

# savefig('mioni_meritve')

# Noise

In [ ]:
from scipy.optimize import curve_fit

def lin_func(t, a, b):
    return a * t + b

In [ ]:
start_fit, end_fit = 80, -1

ks = []

for df in dfs:
    xdata = df['us'].values[start_fit:end_fit]
    ydata = np.log(df['N'].values[start_fit:end_fit])

    popt, pcov = curve_fit(lin_func, xdata, ydata)

    k, k_err = popt[0], np.sqrt(np.diag(pcov))[0]
    ks.append(popt)

In [ ]:
fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)

cc = 0
cs = [['C0', 'C2'], ['C1', 'C3']]
for k, df in zip(ks, dfs):
    xdata = df['us'].values[start_fit:end_fit]
    ydata = np.log(df['N'].values[start_fit:end_fit])
    
    ax.plot(xdata, ydata, lw=1, c=cs[cc][0])
    ax.plot(xdata, lin_func(xdata, *k), lw=1, c=cs[cc][1])
    
    cc += 1

ax.set_xlabel(r'$t$ [$\mu$s]')
ax.set_ylabel('log$N$')
    
# savefig('mioni_fit')

In [ ]:
noise = []
ys = []

for k, df in zip(ks, dfs):
    xdata = df['us'].values[start_fit:end_fit]
    ydata = np.log(df['N'].values[start_fit:end_fit])

    n = ydata - lin_func(xdata, *k)
    
    noise.append(n)

In [ ]:
for i, n in enumerate(noise):
    fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)
    # plt.plot(range(len(n)), n)
    ax.hist(n, bins=int(np.sqrt(len(n))), histtype='step', lw=1, color=f'C{i}')
    ax.set_xlabel(r'šum')
    ax.set_ylabel('$N$')
    # savefig(f'mioni_sum_{i}')

# ECDF: does not work!!!

https://stackoverflow.com/questions/24788200/calculate-the-cumulative-distribution-function-cdf-in-python  
https://en.wikipedia.org/wiki/Probability_integral_transform

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(noise[0])
x, y = ecdf.x, ecdf.y

In [ ]:
plt.plot(x, y)

In [ ]:
from scipy.stats import norm

In [ ]:
def ecdf(a):
    x, counts = np.unique(a, return_counts=True)
    cusum = np.cumsum(counts)
    
    inv = np.argsort(a)
    
    cusum = cusum / cusum[-1]
    
    return x, cusum, inv

In [ ]:
x1, y1, inv1 = ecdf(noise[0])
x2, y2, inv2 = ecdf(noise[1])

In [ ]:
fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)

ax.plot(x1, y1, lw=1)
ax.plot(x2, y2, lw=1)
ax.set_xlabel('šum')
ax.set_ylabel('ECDF')
# savefig('mioni_ecdf')

In [ ]:
plt.hist(y1, bins=15, histtype='step')
plt.hist(y2, bins=15, histtype='step')
plt.show()

In [ ]:
from NIST_tests import RNG_test
from random_helper_functions import float32_to_bin, split_to_arr, bin_str_to_matrix

In [ ]:
from numba import njit

@njit
def binary_tree_walk(arr):
    bits = np.zeros(len(arr)).astype(np.int16)
    for i, a in enumerate(arr):
        if a > 0.5:
            bits[i] = 1
    return bits

In [ ]:
rng1 = y1[inv1]
rng2 = y2[inv2]

rng = np.concatenate((rng1, rng2))
rng = rng[rng != 1]

In [ ]:
bits = float32_to_bin(rng2, cut=9)
bits = ''.join(bits)

result1 = RNG_test(bits, short_df=True)
result1

In [ ]:
bits = binary_tree_walk(rng2).astype(str)
bits = ''.join(bits)

result2 = RNG_test(bits, short_df=True)
result2

CDF ~ U(0, 1) => argsort / max(argsort) ~ U(0, 1) <-> argsort, linspace(0, 1)[argosrt] ~ U(0, 1)

In [ ]:
rngs = [rng1, rng2]

res = []
b = []
for rng in rngs:
    bits = get_bitstring(rng, length=32, cut=30)
    bits = ''.join(bits)
    b.append(bits)

    test_res = RNG_test(bits, short_df=True)
    res.append(test_res)

    
for rng in rngs:
    bits = binary_tree_walk(rng).astype(str)
    bits = ''.join(bits)
    b.append(bits)

    test_res = RNG_test(bits, short_df=True)
    res.append(test_res)

In [ ]:
df = pd.concat([i for i in res])
df.index = [f'$p_{i}$' for i in range(1, len(df)+1)]
df.columns = [i + 1 for i in range(len(df.columns))]

In [ ]:
df

In [ ]:
fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)

bits_arr = split_to_arr(b[0])
m = bin_str_to_matrix(bits_arr)
ax.matshow(m, cmap='Greys_r')
ax.axis('off')
# savefig('mioni_matrika_dobra', save_format='png', dpi=1000)

# Reverse Box–Muller transform

In [ ]:
from scipy.optimize import curve_fit

def gauss(x, a, b, c):
    """c...sigma, b...mu, a...normalization"""
    return a * np.exp(-((x - b)**2) / (2 * c**2))


def make_hist_and_fit(dist, func):
    n_bins = int(np.sqrt(len(dist)))
    y, x = np.histogram(dist, bins=n_bins)
    x = x[1:]
    
    popt, pcov = curve_fit(gauss, x, y)
    return popt, x, y
    

def get_Z(dist):
    popt, _, _ = make_hist_and_fit(dist, gauss)

    sigma, mu = popt[2], popt[1]
    Z = (dist - mu) / sigma
    
    # Z_popt, _, _ = make_hist_and_fit(Z, gauss) # debug
    
    return Z

In [ ]:
import sympy as smp

In [ ]:
X1, X2, U1, U2 = smp.symbols('X1, X2, U_1, U_2', real=True)
R, theta = smp.symbols(r'R, \theta', real=True)

In [ ]:
x1 = R * smp.cos(theta)
x1

In [ ]:
x2 = R * smp.sin(theta)
x2

In [ ]:
eq1 = smp.Eq(X1, x1)
eq1

In [ ]:
eq2 = smp.Eq(X2, x2)
eq2

In [ ]:
res = smp.solve([eq1, eq2], [R, theta], dict=True)
res

In [ ]:
R_, theta_ = res[1][R], res[1][theta]

In [ ]:
R_

In [ ]:
theta_

In [ ]:
eq1 = smp.Eq(R, R_)
eq1

In [ ]:
eq2 = smp.Eq(theta, theta_)
eq2

In [ ]:
eq1 = eq1.subs(R, smp.sqrt(-2 * smp.log(U1)))
eq1

In [ ]:
U1_sol = smp.solve(eq1, U1)[0] # U1
U1_sol

In [ ]:
eq2 = eq2.subs(theta, 2 * smp.pi * U2)
eq2

In [ ]:
U2_sol = smp.solve(eq2, U2)[0] # U2
U2_sol

In [ ]:
U1_f = smp.lambdify([X1, X2], U1_sol)
U2_f = smp.lambdify([X1, X2], U2_sol)

In [ ]:
Z1, Z2 = get_Z(noise[0]), get_Z(noise[1])

n1 = U1_f(Z1, Z2)
n2 = U2_f(Z1, Z2)

from scipy import signal

n2 = n2 + 0.5

In [ ]:
from stat_tests import chi2_test, ks_test

In [ ]:
n = [n1, n2]

tests = []
for i in n:
    tests.append([chi2_test(i, n_bins=20), ks_test(i)])

In [ ]:
tests

In [ ]:
alpha = 0.01

for c, i in enumerate(n):
    fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)
    
    ax.hist(i, histtype='step', color=f'C{c+2}', bins=20)
    
    chi2 = tests[c][0][0][0][0]
    ks = tests[c][1][0][0][0]
    
    critical_value_chi2 = tests[c][0][1]
    critical_value_ks = tests[c][1][1][0]
    
    print(critical_value_ks, critical_value_chi2)
    
    an1 = f'\n$\chi^2$ = {chi2:.2f}, $d$ = {ks:.2e}'
    an2 = f'\n$\chi^2_*$ = {critical_value_chi2:.2f}, $d_*$ = {critical_value_ks:.2e}'
    an3 = r' pri $\alpha$ = {}'.format(alpha)

    if c == 1:
        an = an1 + an2 + an3
    else:
        an = an1

    ax.annotate(an, xy=(0.1, 0.1), xycoords='axes fraction', fontsize=6)

    ax.set_xlabel('$X$')
    ax.set_ylabel('$N$')
    savefig(f'box_muller_uniform_{c}')

In [ ]:
# bits = get_bitstring(n1, length=32)
# bits = ''.join(bits)
bits = binary_tree_walk(n1).astype(str)
bits = ''.join(bits)

res1 = RNG_test(bits, short_df=True)

In [ ]:
# bits = get_bitstring(n2, length=32)
# bits = ''.join(bits)
bits = binary_tree_walk(n2).astype(str)
bits = ''.join(bits)

res2 = RNG_test(bits, short_df=True)

In [ ]:
r = np.concatenate((n1, n2))

# bits = get_bitstring(r, length=32)
# bits = ''.join(bits)
bits = binary_tree_walk(r).astype(str)
bits = ''.join(bits)

res3 = RNG_test(bits, short_df=True)

In [ ]:
res = [res1, res2, res3]

df = pd.concat([i for i in res])
df.index = [f'$p_{i}$' for i in range(1, len(df)+1)]
df.columns = [i + 1 for i in range(len(df.columns))]

In [ ]:
df

In [ ]:
# test
a = np.random.normal(size=100000)
b = np.random.normal(size=100000)

Z1, Z2 = get_Z(a), get_Z(b)

n1 = U1_f(Z1, Z2)
n2 = U2_f(Z1, Z2)

n2 = n2 + 0.5

plt.hist(n1, bins=100)
plt.show()

In [ ]:
print(ks_test(n1))
print(chi2_test(n1, n_bins=100))

In [ ]:
plt.hist(n2, bins=100)
plt.show()

In [ ]:
print(ks_test(n2))
print(chi2_test(n2, n_bins=100))

In [ ]:
bits = get_bitstring(n1, length=32)
bits = ''.join(bits)

RNG_test(bits)

In [ ]:
bits = get_bitstring(n2, length=32)
bits = ''.join(bits)

RNG_test(bits)